# Pipeline for calculating overbite

## Initialization

In [3]:
# Importing packages
import os
import torch
import re
import torchvision.transforms as T
from collections import defaultdict
from PIL import Image

In [1]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  1.13 ; cuda:  cu117
detectron2: 0.6


# Generating png from ply file

In [2]:
%run Pipeline_code/Ply_To_Image.py

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
['Pipeline_data/Ply Files/Brunatest LowerJawScan.ply', 'Pipeline_data/Ply Files/Brunatest_UpperJawScan.ply']


/home/user/app/Pipeline_code/Ply_To_Image.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  vertices = torch.tensor(ply.vertices, dtype=torch.float32)


Brunatest LowerJawScan_0.png: Rotated 90° clockwise_1.png 1/2
Brunatest LowerJawScan_1.png: Flipped on X-axis and rotated 90° clockwise
Brunatest_UpperJawScan_0.png: Rotated 90° clockwise
Brunatest_UpperJawScan_1.png: Flipped on X-axis and rotated 90° clockwise


#  Spltting and flipping of images in Lower/Upper and Left/Right/Middle 

This will only be used when data comes in the form of the data in Raw Data, used for training the model

In [ ]:
# %run Pipeline_code/Opdeling_og_flip_af_billeder.py

# Load the trained model and run on data 

In [ ]:
%run Pipeline_code/Run_model.py

# Pixelmatrix run on data 

In [31]:
%run Pipeline_code/Pixelmatrix.py

100%|███████████████████████████████████████████████████████████████████████| 304/304 [00:26<00:00, 11.63it/s]

Saved refined results to Pipeline_data/Output_after_pixel_matrix/Predicted_keypoints_pixel_matrix.csv
